<a href="https://colab.research.google.com/github/viibhutisharma30/AI_Agent_Based_Research_System/blob/main/AI_Agent_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langgraph openai tavily python-dotenv
!pip install -U langchain-community


ERROR: Could not find a version that satisfies the requirement tavily (from versions: none)
ERROR: No matching distribution found for tavily
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.9 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv


load_dotenv(dotenv_path="text.env")


openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")


print("OpenAI Key:", openai_key[:5] + "****")
print("Tavily Key:", tavily_key[:5] + "****")


OpenAI Key: sk-pr****
Tavily Key: tvly-****


In [ ]:
from langchain.tools.tavily_search import TavilySearchResults


search_tool = TavilySearchResults()

def research_agent(query: str) -> str:
    results = search_tool.run(query)
    return "\n".join([res['content'] for res in results])


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "tiiuae/falcon-rw-1b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

def drafting_agent(research_data: str) -> str:
    prompt = f"Based on the research data, write a detailed answer:\n\n{research_data}"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
pip install langgraph

  Using cached langgraph-0.3.34-py3-none-any.whl.metadata (7.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 22.5 MB/s eta 0:00:00


In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    research_data: str
    answer: str

def research_node(state: AgentState) -> AgentState:
    query = state["query"]
    result = research_agent(query)
    return {"research_data": result}

def drafting_node(state: AgentState) -> AgentState:
    research_data = state["research_data"]
    answer = drafting_agent(research_data)
    return {"answer": answer}

graph_builder = StateGraph(AgentState)
graph_builder.add_node("Research", research_node)
graph_builder.add_node("Draft", drafting_node)
graph_builder.set_entry_point("Research")
graph_builder.add_edge("Research", "Draft")
graph = graph_builder.compile()


In [ ]:
response = graph.invoke({"query": "How is AI used in agriculture?"})
print("✅ Final Answer:\n", response["answer"])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Final Answer:
 Based on the research data, write a detailed answer:

Traditional farming involves various manual processes. Implementing AI models can have many advantages in this respect. By complementing already adopted technologies, an intelligent agriculture system can facilitate many tasks. AI can collect and process big data, while determining and initiating the best course of action. Here are some common use cases for AI in agriculture:
Optimizing automated irrigation systems [...] Application of AI in agriculture provides farmers with real-time crop insights, helping them to identify which areas need irrigation, fertilization, or pesticide treatment. Innovative farming practices such as vertical agriculture can also increase food production while minimizing resource usage. Resulting in reduced use of herbicides, better harvest quality, higher profits alongside significant cost savings.
Automation impact [...] More importantly, the benefits of AI in agriculture are undeniable.

In [ ]:
def remove_duplicates(text: str) -> str:
    lines = text.split("\n")
    seen = set()
    unique_lines = []
    for line in lines:
        if line.strip() not in seen:
            unique_lines.append(line)
            seen.add(line.strip())
    return "\n".join(unique_lines)

cleaned_answer = remove_duplicates(response["answer"])

print("✅ Cleaned Final Answer:\n", cleaned_answer)

✅ Cleaned Final Answer:
 Based on the research data, write a detailed answer:

Traditional farming involves various manual processes. Implementing AI models can have many advantages in this respect. By complementing already adopted technologies, an intelligent agriculture system can facilitate many tasks. AI can collect and process big data, while determining and initiating the best course of action. Here are some common use cases for AI in agriculture:
Optimizing automated irrigation systems [...] Application of AI in agriculture provides farmers with real-time crop insights, helping them to identify which areas need irrigation, fertilization, or pesticide treatment. Innovative farming practices such as vertical agriculture can also increase food production while minimizing resource usage. Resulting in reduced use of herbicides, better harvest quality, higher profits alongside significant cost savings.
Automation impact [...] More importantly, the benefits of AI in agriculture are und

In [ ]:
def refine_query(query: str) -> str:
    vague_keywords = ['tell me', 'how does', 'explain', 'about', 'what is']
    if any(keyword in query.lower() for keyword in vague_keywords):
        refined_query = "Can you tell me more specifically what aspect of AI in agriculture you're interested in? For example, applications, challenges, or technology?"
        return refined_query
    return query  # No need to refine if the query is clear

def interactive_query_adjustments(previous_answer: str) -> str:
    # Ask the user if they need more specific details on the answer
    follow_up_question = input("Do you need more specific details on any part of the answer? Please ask or type 'no' to proceed: ")

    if follow_up_question.lower() != 'no':
        # Refine the query based on user's follow-up
        refined_query = f"Please provide more details on: {follow_up_question}"
        return refined_query
    return previous_answer  # No follow-up, return the previous answer

# Main process
def main():
    # Step 1: User provides an initial query
    query = input("Please enter your query: ")  # Example: "How is AI used in agriculture?"

    # Step 2: Refine the query if it's vague
    refined_query = refine_query(query)
    print(f"Refined Query: {refined_query}")

    # Step 3: Call the research agent with the refined query (simulate agent call)
    response = graph.invoke({"query": refined_query})
    print("✅ Final Answer:\n", response["answer"])

    # Step 4: Ask for follow-up or further clarification
    follow_up_answer = interactive_query_adjustments(response["answer"])
    print(f"Follow-up Answer: {follow_up_answer}")

    # If there's a follow-up, re-query the agent with the refined query (optional)
    if follow_up_answer != response["answer"]:
        # Call the agent with the updated query if needed
        response = graph.invoke({"query": follow_up_answer})
        print("✅ Updated Final Answer:\n", response["answer"])

# Start the main process
main()


Please enter your query: latest AI Improvements towards real-world
Refined Query: latest AI Improvements towards real-world


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Final Answer:
 Based on the research data, write a detailed answer:

LLM executives also spoke about working with foundries to design and make custom silicon, to reduce the costs related to developing features such as recommender systems for ads or videos at scale. Many also said that recent AI advancements—such as continuous learning that enables adaptation based on recent interactions and updates without full retraining—are positive as software and apps proliferate, creating more real-world usage, data exposure and refined training opportunities.
Snowdrop leverages Google Cloud's AI and geospatial data, including Google Places and Vertex AI, to enrich transactional data for financial institutions. This automation has led to a 40% improvement in data accuracy, a 15% increase in merchant-to-transaction matching, and the ability to process over 2.1 billion transactions monthly while scaling globally. [...] *Pix Force is part of the visual inspection sector and is investing in gen AI t